# QURI Chemistry: a Qiskit Function by QunaSys

{/* cspell:ignore QSCI, CCSD, UCCSD, imags, CCGSD, CISD, photoisomerization */}

<LegacyContent>
<Admonition type="note" title="Notes">
* This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/qunasys-quri-chemistry)
* Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan users. They are in preview release status and subject to change.
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note" title="Notes">
* This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/qunasys-quri-chemistry)
* Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan users. They are in preview release status and subject to change.
</Admonition>
</CloudContent>

## Overview

This function helps you solve the quantum chemistry ground state estimation problem by using one of two algorithms based on the [Quantum Selected Configuration Interaction (QSCI) algorithm](https://arxiv.org/pdf/2302.11320):

- QSCI
- Optimization-based QSCI

The overall procedure for using this function is summarized in the following flow chart:

![Workflow](/images/guides/qunasys-qsci/workflow.svg)

## Function description

The QSCI algorithm samples from a specific ansatz on quantum computers. Each sampled bitstring corresponds to an electron configuration, and the number of times a bitstring is sampled represents the importance of that electron configuration. Choose a number, $R$, to select the $R$ most important electron configurations from the sample. The function constructs and diagonalizes the subspace Hamiltonian, depending on your choice of $R$. The smallest eigenvalue is called "QSCI energy" and is the estimation of the QSCI algorithm's true ground state energy.

One important QSCI ingredient is its initial state preparations. Several possible initial state preparations are provided in this function. These include the [hardware efficient](https://www.nature.com/articles/nature23879), [UCCSD](https://www.nature.com/articles/srep03589), [kUpCCGSD](https://arxiv.org/abs/1810.02327), and [kuCJ](https://arxiv.org/abs/1909.12410) ansatzes. A CCSD amplitude-based double excitation ansatz is provided specifically to be executed on near-term devices. Ansatz-specific settings are also provided for you to customize according to your needs.

In addition to returning the QSCI energy, the function also computes the estimated ground state wave function to help you identify the important electron configurations.

## Get started
<LegacyContent>
First, authenticate using your [IBM Quantum&reg; API key](http://quantum.ibm.com/) and select the Qiskit Function as follows:
</LegacyContent>
<CloudContent>
First, authenticate using your [IBM Quantum&reg; API key](http://quantum.cloud.ibm.com/) and select the Qiskit Function as follows:
</CloudContent>

In [ ]:
from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog()

function = catalog.load("qunasys/quri-chemistry")

## Inputs

The function is called with the following arguments:

```
function.run(
    method = ...,               # Allow only "QSCI" or "OPT_QSCI"
    molecule = ...,             # The molecule
    circuit_options = {...},    # Options for customizing the circuits
    qsci_setting = {...},       # Parameters of the QSCI algorithm
    mitigation_setting = {...}  # Specifies the error mitigation method
    max_iter = ...              # Maximum number of iterations when method = "OPT_QSCI".
    instance = ...              # The instance to use.
    backend_name = ...          # The backend name to use.
)

```

Each argument is described in following table.

<LegacyContent>
| Name               | Type           | Description                                                                                      | Required | Default                      | Example                                      |
| :----------------- | :------------- | :----------------------------------------------------------------------------------------------- | :------- | :--------------------------- | :------------------------------------------- |
| method             | str            | The algorithm name                                                                               | Yes      | -                            | “QSCI” or "OPT_QSCI"                         |
| mole               | json           | The molecule setting.                                                                            | Yes      |                              | The molecule setting. See the examples that follow the table.  |
| circuit_options    | json           | The settings for the circuit.                                                                     | Yes      |                              | See the examples following the table.                         |
| qsci_setting       | dict[str, int] | Specify the number of shots and the size of the subspace. See the following section for details. | Yes      |                              | See the QSCI setting table                   |
| mitigation_setting | json           | Settings for configuring error mitigation.                                                        | No       | None                         | See the mitigation section.             |
| max_iter           | int            | The maximum number of iterations when the method is "OPT_QSCI"                                    | No       | 2000                         | 5                                            |
| backend_name       | str            | The backend name to use                                                                 | No       | The least busy one available | “ibm_torino”                                 |
| instance           | str            | The instance to use                                                                     | No       | "hub1/group1/project1"       |                                              |
</LegacyContent>
<CloudContent>
| Name               | Type           | Description                                                                                      | Required | Default                      | Example                                      |
| :----------------- | :------------- | :----------------------------------------------------------------------------------------------- | :------- | :--------------------------- | :------------------------------------------- |
| method             | str            | The algorithm name                                                                               | Yes      | -                            | “QSCI” or "OPT_QSCI"                         |
| mole               | json           | The molecule setting.                                                                            | Yes      |                              | The molecule setting. See the examples that follow the table.  |
| circuit_options    | json           | The settings for the circuit.                                                                     | Yes      |                              | See the examples following the table.                         |
| qsci_setting       | dict[str, int] | Specify the number of shots and the size of the subspace. See the following section for details. | Yes      |                              | See the QSCI setting table                   |
| mitigation_setting | json           | Settings for configuring error mitigation.                                                        | No       | None                         | See the mitigation section.             |
| max_iter           | int            | The maximum number of iterations when the method is "OPT_QSCI"                                    | No       | 2000                         | 5                                            |
| backend_name       | str            | The backend name to use                                                                 | No       | The least busy one available | “ibm_torino”                                 |
| instance           | str            | The cloud resource name of the instance to use                                                                     | No       | "CRN"       |                                              |

</CloudContent>
<Admonition type="note">
Refer to the [Specify an instance in your code](/guides/instances#specify-an-instance-in-your-code) guide to learn more.
</Admonition>

### `molecule`

Specify the details about the molecule here. The input is the same as the `pyscf.gto.M` with an additional `active_space` option. Detailed settings for configuring a molecule are summarized in the following table.

| Name         | Type  | Description                                                                                                                                                      | Required | Default  | Example                                                 |
| :----------- | :---- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------- | :------- | :------------------------------------------------------ |
| atom         | str   | The list of atom coordinates                                                                                                                                     | Yes      | -        | “H 0 0 0; H 0 0 1”                                      |
| basis        | str   | The basis set to represent the electronic wave function. Allowed values are the same as those supported by [PySCF](https://pyscf.org/_modules/pyscf/gto/basis.html) | No       | “sto-3g” | “sto-3g”                                                |
| spin         | float | The $s_z$ quantum number of the molecule. Only spin=0.0 is supported                                                                                            | No       | 0.0      | 0.0                                                     |
| charge       | int   | The total charge of the molecule                                                                                                                                | No       | 0        | 0                                                       |
| active_space | json  | The active space you want to choose. Review the “Active space” table for more information                                                                     | No       | None     | Review the “Active space” table for more information |


The active space settings are summarized below.

| Name                | Type      | Description                                | Required | Default | Example   |
| :------------------ | :-------- | :----------------------------------------- | :------- | :------ | :-------- |
| n_active_ele        | int       | The number of active electrons             | Yes      | -       | 4         |
| n_active_orb        | int       | The number of active spatial orbitals      | Yes      | -       | 3         |
| active_orbs_indices | list[int] | The list of active spatial orbital indices | No       | None    | [0, 1, 2] |

### The `circuit_options` field

The `circuit_options` field should be a dictionary containing detailed settings of the circuit for running the QSCI algorithm - for example, the state preparation settings. In general it should take this form:
```
"circuit_options": {
    "ansatz": ...,
    "state_prep_method": ...,
    "ansatz_setting": ...,
}
```

| Name              | Type | Description                                                                                                                                                                                                     | Required | Default | Example                |
| :---------------- | :--- | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------- | :------ | :--------------------- |
| ansatz            | str  |  The name of the ansatz to use. Only “DoubleExcitation” and “UCCSD” are supported. `DoubleExcitation` is highly recommended to obtain a stable result, especially for systems larger than eight qubits. | Yes      |         | “DoubleExcitation” | Yes      |         | “DoubleExcitation”     |
| state_prep_method | str  | How to prepare the circuit parameter of the ansatz. Only CCSD is allowed.                                                                                                                          | No       | “CCSD"  | “CCSD”                 |
| ansatz_setting    | str  | Specific setting to configure the ansatz with                                                                                                                                                         | No       | None    | See the examples following the table. |




Each field is explained in the subsequent sections.

#### `ansatz`, `state_prep_method` and `ansatz_setting`

The `ansatz` string specifies the ansatz to use. This specifies a type of parametric circuit without specifying what the concrete circuit parameters are.
`state_prep_method` specifies how the circuit parameter is prepared. Finally, the `ansatz_setting` lets you  customize the ansatz.

| Ansatz             | Preparation method     | Ansatz Setting Default                                      | Note                                                   |
| ------------------ | ---------------------- | ----------------------------------------------------------- | ------------------------------------------------------ |
| Double Excitation  | CCSD                   | `{n_amplitudes: 10}`                                        | Lets you optimize from the result in the last example. |
| UCCSD              | CCSD (default), Random | `{use_singles: True, n_trotter: 1, reduce_parameter: True}` | Not recommended for systems with eight or more qubits. |
| KuCJ               | Random                 | `{k: 1}`                                                    | Recommended for system up to 16 qubits                 |
| KUpCCGSD           | Random                 | `{k: 1, n_trotter: 1, reduce_parameter: False} `            |                                                        |
| Hardware Efficient | Random                 | `{n_layers: depends on qubit size}        `                 |                                                        |


<Admonition type="caution">
When `method` is "QSCI", only the "CCSD" preparation method is supported. The "Random" preparation method is only supported when method = "OPT_QSCI".
</Admonition>

<Admonition type="note">
Detailed settings for UCCSD ansatz can be found in the [QURI Parts documentation](https://quri-parts.qunasys.com/api/quri_parts/openfermion/quri_parts.openfermion.ansatz.uccsd#quri_parts.openfermion.ansatz.uccsd.TrotterUCCSD). The `reduce_parameter` option is the same as the `singlet_excitation` option in the QURI Parts documentation.
</Admonition>

<Admonition type="note">
Detailed settings for KUpCCGSD ansatz can be found in the [QURI Parts documentation](https://quri-parts.qunasys.com/api/quri_parts/openfermion/quri_parts.openfermion.ansatz.kupccgsd#quri_parts.openfermion.ansatz.kupccgsd.KUpCCGSD).
</Admonition>

### `qsci_setting`


| Name                      | Type | Description                                                                                                                                                                                                                                                                                                          | Required | Example |
| :------------------------ | :--- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------- | :------ |
| n_shots                   | int  | The total number of shots to sample from the ansatz                                                                                                                                                                                                                                                                  | Yes      | 100000  |
| number_of_states_pick_out | int  | The size of the subspace Hamiltonian is $R$. It affects the _classical_ resource, where _classical_ diagonalization of the $R \times R$ subspace Hamiltonian is performed if the number of bitstrings ($n$) sampled from the ansatz is larger than $R$. Otherwise, the $n \times n$ subspace matrix is diagonalized. | Yes      | 10000   |

### `mitigation_setting`

Usually, the problem at hand needs to respect some symmetry, especially particle number and spin conservation. To restore symmetry, use symmetry post-selection or [configuration recovery](https://arxiv.org/pdf/2405.05068v1) error mitigation. You can choose either of them with the `mitigation_setting` field in either algorithm. Symmetry post-selection is the default value.

`post-selection` removes the erroneous states completely, leaving only the basis states to construct the subspace Hamiltonian. This usually saves classical resources, but might miss many states. Configuration recovery attempts to flip individual bits in erroneous basis states with respect to some probability distribution. Using configuration recovery increases the number of basis states to build a larger subspace Hamiltonian, but it uses a lot more classical resources.

The options are explained in the following table.

| Name                     | Type                        | Description                                                                                                                              | Required | Default |
| :----------------------- | :-------------------------- | :--------------------------------------------------------------------------------------------------------------------------------------- | :------- | :------ |
| particle_number_symmetry | bool                        | If True, remove bitstrings that do not respect particle number conservation from sampling result.                                        | No       | True    |
| spin_symmetry            | bool                        | If True, remove bitstrings that do not respect $s_z$ conservation from the sampling result.                                              | No       | True    |
| configuration_recovery   | Union[bool, dict[str, int]] | Activate configuration recovery if set to a value other than False. Refer to the configuration recovery section for dict[str, int] input | No       | False   |

<Admonition type="caution">
You cannot set spin symmetry to True when particle symmetry is False.
</Admonition>


#### Symmetry post-selection

These keys control the symmetry post-selection: `particle_number_symmetry` and `spin_symmetry`. Setting `particle_number_symmetry` to True selects those states that conserve the total electron numbers.  Setting `spin_symmetry` to True selects those states that conserve the $s_z$ quantum number. Currently only $s_z = 0$ is supported. The default is:

In [2]:
mitigation_setting = {
    "particle_number_symmetry": True,
    "spin_symmetry": True,
}

#### Configuration recovery

Instead of the symmetry post-selection, you can choose to do configuration recovery. To turn it on, set it to `True`, and optionally customize it with a dictionary containing the following parameters:


| Name                      | Type        | Description                                                                                                          | Required | Default |
| :------------------------ | :---------- | :------------------------------------------------------------------------------------------------------------------- | :------- | :------ |
| n_recover_iteration       | int         | The number of recovery iterations.                                                                                   | No       | 5       |
| n_batch                   | int         | The number of batches generated per iteration.                                                                       | No       | 1       |
| number_of_states_pick_out | int or None | The dimension of the QSCI vector in the recovery process. If None, it is set to the same value that was used for QSCI. | No       | None    |
| recovery_r_multiple       | int         | The multiplier for number_of_states_pick_out. This is the number of states selected from the raw sample.             | No       | 10      |

In [3]:
# Use the default configuration recovery setting
default_mitigation_setting = {"configuration_recovery": True}

# Customized configuration recovery setting
customized_mitigation_setting = {
    "configuration_recovery": {
        "n_recover_iteration": 20,
        "n_batch": 10,
        "number_of_states_pick_out": 5000,
    }
}

<Admonition type="caution">
Other error mitigations are not allowed when configuration recovery is turned on.
</Admonition>

## Outputs

The output of the function contains these fields:

| Name         | Type  | Description                                                             |
| ------------ | ----- | ----------------------------------------------------------------------- |
| qsci_energy  | float | The QSCI energy that acts as the estimation of the ground state energy. |
| state_vector | dict  | A dictionary representing the QSCI estimated ground state vector.       |

The `state_vector` field is a dictionary containing these fields:

| Name  | Type        | Description                                                                                    |
| ----- | ----------- | ---------------------------------------------------------------------------------------------- |
| bits  | list[int]   | A list of integers representing the electron configurations                                   |
| reals | list[float] | Real parts of the amplitudes of the electron configurations at the corresponding positions      |
| imags | list[float] | Imaginary parts of the amplitudes of the electron configurations at the corresponding positions |

## Example

This example shows you how to compute the ground state energy of the water molecule using the QSCI algorithm.

### Set up the molecule

First, configure the setting for a $H_2O$ molecule in JSON format. The coordinates of the water molecule can be loaded from [OpenFermion](https://quantumai.google/reference/python/openfermion/chem/geometry_from_pubchem).

In [4]:
molecule = {
    "atom": "O 0 0 0; H 0.2774 0.8929 0.2544; H 0.6068 -0.2383 -0.7169",
    "basis": "6-31g",  # default to "sto-3g"
    "spin": 0.0,  # default to 0.0. Current code forces spin = 0.0
    "charge": 0,  # default to 0
    "active_space": {  # default to None. Equivalent to no active space chosen.
        "n_active_ele": 10,
        "n_active_orb": 10,
        "active_orbs_indices": [i for i in range(10)],
    },
}

### Execute algorithms

All the function input is in JSON format. You will set up a JSON string that specifies the options for running an algorithm. As stated previously, the two types of algorithms provided are "QSCI" and "OPT_QSCI".

#### QSCI

First, set up the number of shots and the maximum subspace Hamiltonian size with the `qsci_setting` field.

In [94]:
qsci_setting = {"n_shots": 1e5, "number_of_states_pick_out": 12000}

Next, set up the ansatz. This example uses the `DoubleExcitation` as the ansatz because it carries the most non-trivial information with the lowest circuit depth. You can specify how many excitation amplitudes to include with the `n_amplitudes` key in the `ansatz_setting` field. The default value is 10. The function sorts the CCSD amplitudes according to their magnitude and chooses the largest `n_amplitudes` to construct the ansatz. An example JSON dictionary using this ansatz follows:

In [116]:
qsci_double_exc_json = {
    "ansatz": "DoubleExcitation",
    "state_prep_method": "CCSD",  # This is optional with a default of "CCSD".
    "ansatz_setting": {
        "n_amplitudes": 20
    },  # This is optional with a default of {"n_amplitudes": 10}.
}

Turn on the configuration recovery algorithm for error mitigation.

In [ ]:
mitigation_setting = {  # Refer to the "Error mitigation" section for details.
    "configuration_recovery": {"number_of_states_pick_out": 10000}
}

Finally, execute the function.

In [ ]:
# This cell is hidden from users
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
instance = service.active_account()["instance"]
backend_name = service.least_busy().name

In [ ]:
job = function.run(
    method="QSCI",
    molecule=molecule,
    circuit_options=qsci_double_exc_json,
    qsci_setting=qsci_setting,
    mitigation_setting=mitigation_setting,
    instance=instance,
    backend_name=backend_name,  # E.g. "ibm_torino"
)

Print out the result from the run:

In [125]:
result = job.result()
print(result)

{'qsci_energy': -76.05069405129802, 'state_vector': {'bits': [1023, 131583, 66303, 2943, 9087, 9183, 5103, 3039, 9207, 525183, 5055, 525279, 263103, 2031, 3063, 33663, 263151, 133503, 1983, 17391, 139647, 33759, 17343, 5115, 525303, 12543, 6399, 133599, 655743, 139743, 2043, 68079, 196863, 655839, 164223, 3327, 33783, 263163, 264447, 24831, 133623, 164319, 528639, 11103, 70383, 141663, 527199, 37119, 68031, 9471, 18687, 74223, 270591, 328383, 17403, 70335, 67263, 533343, 135663, 139767, 786687, 74175, 655863, 68223, 74367, 14463, 67311, 540927, 525567, 11127, 135615, 328431, 663903, 590271, 6975, 49407, 11223, 590319, 82671, 164343, 35679, 13263, 13215, 70395, 132591, 132543, 98799, 135903, 393663, 295167, 135807, 41823, 68091, 6063, 172383, 657759, 267183, 536703, 7023, 13167, 557919, 132831, 74487, 147903, 7119, 99039, 74463, 82623, 13119, 166239, 3903, 14559, 393855, 3999, 527319, 197583, 529359, 268479, 529263, 264111, 37743, 147951, 35703, 68319, 530655, 265023, 527223, 271263, 59

Use the \"qsci_energy\" key to see the QSCI energy:

In [126]:
CASCI_ENERGY = -76.0508056368444  # Exact result
qsci_energy = result["qsci_energy"]
print(f"CASCI energy: {CASCI_ENERGY} Ha")
print(f"QSCI energy: {qsci_energy} Ha")
print(f"Energy error: {(qsci_energy - CASCI_ENERGY) * 1000: .2e} mHa")

CASCI energy: -76.0508056368444 Ha
QSCI energy: -76.05069405129802 Ha
Energy error:  1.12e-01 mHa


The returned ground state energy estimation of an $\text{H}_2 \text{O}$ molecule is very close to the exact result!

The `state_vector` field contains three values: `bits`, `reals`, and `imags`. The `bits` value is a list of integers representing electron configurations. The $i$-th position of the `reals` (`imags`) field represents the real (imaginary) part of the amplitude of the electron configuration on position $i$. You can sort it by magnitude and print out the top 10 most important electron configurations.

In [76]:
import numpy as np

amps = np.array(result["state_vector"]["reals"]) + 1j * np.array(
    result["state_vector"]["imags"]
)
sort_idx = np.argsort(np.abs(amps))

for i, (bit, real, imag) in enumerate(
    zip(
        np.array(result["state_vector"]["bits"])[sort_idx][::-1],
        np.array(result["state_vector"]["reals"])[sort_idx][::-1],
        np.array(result["state_vector"]["imags"])[sort_idx][::-1],
    )
):
    if i > 20:
        break
    print(
        f"Electron configuration: {bin(bit)[2:].zfill(20)}, "
        f"amplitude: {real + 1j * imag: .3e}, "
        f"magnitude: {real**2 + imag**2: .3e}"
    )

Electron configuration: 00000000001111111111, amplitude: -6.114e-01+7.731e-01j, magnitude:  9.715e-01
Electron configuration: 00110000000011111111, amplitude:  3.561e-02-4.503e-02j, magnitude:  3.296e-03
Electron configuration: 00000011001111001111, amplitude:  2.643e-02-3.341e-02j, magnitude:  1.815e-03
Electron configuration: 00001100001111001111, amplitude:  1.789e-02-2.262e-02j, magnitude:  8.315e-04
Electron configuration: 00100001000111101111, amplitude: -1.717e-02+2.171e-02j, magnitude:  7.662e-04
Electron configuration: 00010010001011011111, amplitude: -1.713e-02+2.166e-02j, magnitude:  7.623e-04
Electron configuration: 00000001101101101111, amplitude: -1.577e-02+1.994e-02j, magnitude:  6.463e-04
Electron configuration: 00000010011110011111, amplitude: -1.575e-02+1.992e-02j, magnitude:  6.450e-04
Electron configuration: 11000000001111001111, amplitude:  1.548e-02-1.957e-02j, magnitude:  6.228e-04
Electron configuration: 00000000111100111111, amplitude:  1.536e-02-1.942e-02j, ma

#### Optimization-based QSCI

If you don't want to use CCSD as the state preparation method, or if you want to optimize further from the CCSD initial state, you can use the optimization-based QSCI. This algorithm gives you access to more chemistry-inspired ansatz with random initial parameters. For this algorithm, the QSCI energy is used as the cost function for the COBYLA optimizer. In each iteration, the circuit parameters are updated and the QSCI energy is evaluated with the circuit carrying the new set of circuit parameters.

The JSON dictionary used to run this algorithm is similar to that of QSCI. It only contains one additional field, "max_iter", which lets you limit the resource usage. The default is 2000 iterations.

Example of running this algorithm with the 1-uCJ ansatz:

In [ ]:
opt_qsci_1ucj_json = {
    "ansatz": "KuCJ",
    "state_prep_method": "RANDOM",  # This is optional with a default of "RANDOM" for "KuCJ".
    "ansatz_setting": {
        "k": 1
    },  # This is optional with the default described in the previous table.
}

In [ ]:
opt_job = function.run(
    method="OPT_QSCI",
    molecule=molecule,
    parameters=opt_qsci_1ucj_json,
    qsci_setting={"n_shots": 1e5, "number_of_states_pick_out": 5e4},
    mitigation_setting={
        "configuration_recovery": {"number_of_states_pick_out": 10000}
    },
    max_iter=5,
    instance=instance,
    backend_name=backend_name,  # E.g. "ibm_strasbourg"
)

Print out the result.

In [ ]:
opt_result = opt_job.result()
print(opt_result)

{'qsci_energy': -76.05051672600099, 'state_vector': {'bits': [1023, 1983, 2943, 33783, 3039, 263151, 33663, 2031, 5103, 9087, 525183, 33759, 2043, 17343, 9207, 9213, 525279, 17406, 131583, 5115, 525303, 263163, 525309, 17391, 9183, 3063, 263103, 66303, 3069, 2046, 17403, 263166, 5118, 33789, 5055, 67323, 133503, 70335, 264123, 655863, 67263, 279471, 139743, 6138, 67311, 655743, 133599, 139647, 11127, 164223, 267195, 527223, 18411, 18426, 533343, 558039, 82671, 21486, 264111, 267183, 11223, 328443, 328446, 70383, 328431, 35799, 533373, 6063, 527199, 11103, 6126, 21483, 533469, 18363, 279534, 67326, 41847, 533463, 279486, 655839, 533367, 133629, 70398, 11229, 21498, 558045, 11133, 82683, 21423, 41823, 139767, 527319, 35679, 557919, 264126, 164343, 10095, 35703, 267243, 655869, 267198, 21435, 328383, 279531, 527325, 271356, 6075, 18366, 83631, 164349, 264171, 557943, 271311, 83691, 139773, 527229, 6078, 133623, 13215, 82686, 6123, 68319, 41949, 279546, 35805, 21438, 18414, 270591, 279483,

The returned result has the same fields as the one in the QSCI example.

In [91]:
CASCI_ENERGY = -76.0508056368444  # Exact result
opt_qsci_energy = opt_result["qsci_energy"]
print(f"CASCI energy: {CASCI_ENERGY} Ha")
print(f"QSCI energy: {opt_qsci_energy} Ha")
print(f"Energy error: {(opt_qsci_energy - CASCI_ENERGY) * 1000: .2e} mHa")
print()
print("Top 20 important configurations:")
amps = np.array(opt_result["state_vector"]["reals"]) + 1j * np.array(
    opt_result["state_vector"]["imags"]
)
sort_idx = np.argsort(np.abs(amps))

for i, (bit, real, imag) in enumerate(
    zip(
        np.array(opt_result["state_vector"]["bits"])[sort_idx][::-1],
        np.array(opt_result["state_vector"]["reals"])[sort_idx][::-1],
        np.array(opt_result["state_vector"]["imags"])[sort_idx][::-1],
    )
):
    if i > 20:
        break
    print(
        f"Electron configuration: {bin(bit)[2:].zfill(20)}, "
        f"amplitude: {real + 1j * imag: .3e}, "
        f"magnitude: {real**2 + imag**2: .3e}"
    )

CASCI energy: -76.0508056368444 Ha
QSCI energy: -76.05051672600099 Ha
Energy error:  2.89e-01 mHa

Top 20 important configurations:
Electron configuration: 00000000001111111111, amplitude: -1.934e-01-9.664e-01j, magnitude:  9.713e-01
Electron configuration: 00110000000011111111, amplitude:  1.127e-02+5.629e-02j, magnitude:  3.295e-03
Electron configuration: 00000011001111001111, amplitude:  8.434e-03+4.214e-02j, magnitude:  1.847e-03
Electron configuration: 00001100001111001111, amplitude:  5.640e-03+2.818e-02j, magnitude:  8.258e-04
Electron configuration: 00010010001011011111, amplitude: -5.445e-03-2.721e-02j, magnitude:  7.698e-04
Electron configuration: 00100001000111101111, amplitude: -5.418e-03-2.707e-02j, magnitude:  7.622e-04
Electron configuration: 00000010011110011111, amplitude: -5.053e-03-2.524e-02j, magnitude:  6.628e-04
Electron configuration: 00000001101101101111, amplitude: -5.023e-03-2.510e-02j, magnitude:  6.551e-04
Electron configuration: 00000000111100111111, amplit

## Performance

### N2 dissociation curve

As a performance benchmark, the 20-qubit $\text{N}_2$ dissociation curve is shown.

![N2_dissociation](/images/guides/qunasys-qsci/N2_dissociation.svg)

The red curve in the plot is generated by the QSCI method with the `DoubleExcitation` ansatz. You can reproduce it by using the following code with different choices of `d`, the distance between the nitrogen atoms.

In [ ]:
d = 1.0

job = function.run(
    method="QSCI",
    molecule={"atom": f"N 0 0 0; N 0 0 {d}"},
    parameters={"ansatz": "DoubleExcitation", "state_prep_method": "CCSD"},
    qsci_setting={"n_shots": 100000, "number_of_states_pick_out": 50000},
    mitigation_setting={"configuration_recovery": True},
    instance=instance,
    backend_name=backend_name,  # E.g. "ibm_strasbourg"
)

Note that each point typically takes about one minute of QPU usage. On the classical side, where $R$ is chosen to be 50000, it doesn't necessarily mean that diagonalization of a $50000 \times 50000$ matrix is done. The number of samples after configuration recovery in this case is approximately 5000 - 7000, which sets the dimension of the subspace Hamiltonian. Thus, the classical diagonalization cost is much smaller than that of typical FCI computation.

The key takeaway of this result is that QSCI with suitable configuration recovery outperforms scalable classical methods such as Hartree-Fock (HF), CCSD, and CISD at _all_ choices of `d`. This implies that at a scale where FCI energy is no longer available, QSCI serves as a reliable method to estimate the ground state energy at all distances. For example, using the 6-31g basis,
a 36-qubit $\text{N}_2$ dissociation curve produced by QURI Function QSCI is given by

![N2_dissociation](/images/guides/qunasys-qsci/36_qubit_N2_dissociation.svg)

Here, the FCI curve is not available on a typical laptop (M2 Pro Chip, 16G RAM). However, QSCI can produce a qualitatively correct dissociation curve compared to the classical methods on the same machine with the assistance of the `ibm_strasbourg` quantum computer.

### Azobenzene

Azobenzene has two isomers, trans-azobenzene and cis-azobenzene. The energy difference between the ground states of the two isomers plays an important role in the photoisomerization of azobenzene. This example benchmarks the ground state energy difference between QSCI and FCI with different active space settings up to 28 qubits. With configuration recovery, QURI Function QSCI yields error less than 5 mHa with only $10^5$ shots, 30 to 40 seconds of QPU usage, and under 10 minutes of classical post-processing time per point.

![Azobenzene](/images/guides/qunasys-qsci/azobenzene.svg)

## Support

For running systems larger than 20 qubits, please contact [sales@qunasys.com](mailto:sales@qunasys.com).

## Next steps

<Admonition type="tip" title="Recommendations">

- [Request access to QunaSys QURI Chemistry.](https://quantum.ibm.com/functions?id=42f5a0ea-2c74-4681-a973-3ef97de97ee4)
- Try the [Compute dissociation curves for strong coupling systems with QunaSys QURI Chemistry](https://learning.quantum.ibm.com/tutorial/compute-dissociation-curves-for-strong-coupling-systems-with-quna-sys-qsci) tutorial.

</Admonition>